## tinyFrame Analysis - General Calculations

## Purpose
This document serve as the repository for all analysis for the tinyFrame project.

## TODO:
- ~~Calculate required deflection~~
- ~~Build calculations around known press-force (7 lbf)~~
- ~~stress checks for given thickness and the declarations above~~
- ~~Calculate combined stress case due to additional torsion created by double-back feature (snake)~~
- Calculate 
- Create a plot that shows:
  - Load
  - Length
  - Gauge

In [1]:
import numpy, yaml, pint
import numpy as np
import matplotlib.pyplot as plt

# Boilerplate
## Initialize Pint Registry for Unit Manipulation
## See documentation on Registries and Units in Pint here:
## https://pint.readthedocs.io/en/stable/getting/tutorial.html
from pint import UnitRegistry
ureg = UnitRegistry(auto_reduce_dimensions=True)

In [2]:
preferred_units = [
   ureg.m,  # distance      L
   ureg.kilogram,  # mass          M
   ureg.s,  # duration      T
   ureg.c,  # temperature   Θ
   ureg.newton,  # force         L M T^-2
   ureg.W,  # power         L^2 M T^-3
]

Equation that describes deflection as a function of loading for wrap:
$$\delta = \frac{Pl^3}{3EI}$$

Equation for Second Moment of Area of a Rectangular beam:
$$I = \frac{bh^3}{12}$$

Before solving for the required locking tab length, we first need to find the prototyped deflection to determine the required length in thicker gauge configurations of the tubing system.

Solving the equation above and replacing the second moment of area, I with the locking tab dimensions, we get:

$$\delta = \frac{4 P L^3}{E b h^3}$$

We normalize to the number of tabs in the system (four) and end up with:


$$\delta = \frac{P L_{protoyped}^3}{E b h^3}$$




In [94]:
# Values determined from Prototyping (Thanks Jonathan & Bryce!)
E = 69e9 * ureg.newton / (ureg.meter * ureg.meter) # Pa - young's modulus
P = 31 * ureg.newton # From Prototyping
L_prototyped = 75 * ureg.mm # From CAD

# Second moment of Area for Beam
b = 4 * ureg.mm
h = 1.651 * ureg.mm

In [95]:
del_ = (P * L_prototyped**3) / (E * b * h**3)

print("Distance for Locking Flexure to deflect for installation: ")
print(del_.to(ureg.mm))

Distance for Locking Flexure to deflect for installation: 
10.529195322197207 millimeter


Given the analysis matches CAD, we can utilize the required deflection as a function of the tab depth as opposed to what was regressed from the calculations above. That will be declared as del_tab.

We need to calculate the length of the flexure (locking tabs) required - solve the above equations for I:

$$L = \sqrt[3]{\frac{3 \delta \, E \, b \, h^3}{12P}}$$

P represents the global press force, but we assume there are always four tabs, so we divide P by 4 and are left with:

$$L = \sqrt[3]{\frac{\delta \, E \, b \, h^3}{P}}$$

In [96]:
# Declare variables required:
del_tab = 8 * ureg.mm # From CAD


# Determine the length of the locking tabs
L = ((del_tab * E * b * h**3) / (P)) ** (1/3)

#print(L_temp.to(ureg.mm))
print(L)

68.43729963070291 millimeter


Now, we check for bending stress:

$$\sigma_b = \frac{M \cdot c}{I}$$

In [97]:
# Declare variables
sigma_yield = 73 * ureg.ksi
c = h/2
I = b*h**3 / 12

# Calculate requisite components
M = P/4 * L

sigma_bending =  M * c / I

# What's the factor of safety against the material yield stress?
FOS_install = sigma_yield / sigma_bending

print("Bending stress seen by the locking tab: ")
print(sigma_bending.to(ureg.ksi))

print("Factor of safety against yield: ")
print(FOS_install)

Bending stress seen by the locking tab: 
42.3323810907579 kip_per_square_inch
Factor of safety against yield: 
1.7244482384180735 dimensionless


# Checking for Combined Loading of Torsional and Bending Stresses
The second polar moment of area for a rectangular beam is:
$$J \approx \beta a b^3$$

Where:

a is the length of the long side

b is the length of the short side

β is the torsional constant


First, calculate ratio a/b and declare it as W to pass to a function to return torsional constant $\beta$.

In [98]:
# Variables for Torsional Stress Calculations
# Note: Normalizing P (load) to P_ to delinate load distributed over four locking tabs
# e is the hypotenuse of the b/h triangle: 
e = 5 * ureg.mm
r = (b**2 / 4 + h**2 / 4)**(1/2)
P_ = P / 4

# Calculations
T = P_ * e

#Known data points
W_values = np.array([1.0, 1.5, 2.0, 2.5, 3.0, 4.0, 5.0, 6.0, 10.0])
beta_values = np.array([0.141, 0.196, 0.229, 0.249, 0.263, 0.281, 0.291, 0.299, 0.312])

def get_beta(W):
    """
    Returns the interpolated beta value for a given W.
    If W is outside the data range, it will extrapolate.
    """
    return float(np.interp(W, W_values, beta_values))

    
# Equations for second polar moment of area have an underscore suffix
a_ = b
b_ = h
W = a_/b_

# Calculate Torsional Constant
beta = float(np.interp(W, W_values, beta_values))

# Calculate Second Polar Moment of Area
J = beta * a * b**3

# Torsional Stress is Tr/J
tau_max = T * r / J



# Finally, we calculate the combined stress
Pulled from the equivalent von mises stress theory, reference here: https://www.continuummechanics.org/vonmisesstress.html
$$\sigma_v = \sqrt{\sigma^2 + 3\tau^2}$$


In [99]:
# Equivalent Stress
sigma_v = (sigma_bending**2 + 3*tau_max**2)**(1/2)

print(sigma_v.to(ureg.MPa))
print(sigma_v.to(ureg.ksi))

291.8750344281311 megapascal
42.332894693383054 kip_per_square_inch
